# Functional Programming in Python

even thougth python is pretty OOP, some useful FP constructs can be hacked into our code

### python mantra

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
from __future__ import print_function

## FP mantra

don't think about how to do something, think about the **transformation**

## Standard python tools for some basic functional programming 

**map filter reduce!**

In [3]:
map(lambda x: x**2, range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [4]:
filter(lambda y: y % 2 == 0, map(lambda x: x**2, range(10)))

[0, 4, 16, 36, 64]

In [5]:
reduce(lambda a,i: a+i, filter(lambda y: y % 2 == 0, map(lambda x: x**2, range(10))))

120

### list comprehensions (from Haskell) are a bit more consise

In [6]:
[x**2 for x in range(10) if x %2 ==0]

[0, 4, 16, 36, 64]

In [7]:
# more compact version of result above
sum(x**2 for x in range(10) if x %2 ==0)

120

### use some other functions as args

In [8]:
zip('ABC',[1,2,3])

[('A', 1), ('B', 2), ('C', 3)]

In [9]:
dict(zip(['A','B','C'],[1,2,3]))

{'A': 1, 'B': 2, 'C': 3}

In [10]:
dict(zip('ABC',[1,2,3]))

{'A': 1, 'B': 2, 'C': 3}

### itertools

https://docs.python.org/3/howto/functional.html

### decorators

these are useful for creating functional behaviour in python

In [12]:
def my_decorator(some_function):
    def wrapper():
        print("*** Before some_function() is called.")
        some_function()
        print("*** After some_function() is called.")
    return wrapper

@my_decorator
def just_some_function():
    print('\tSome amazing function')

In [13]:
just_some_function()

*** Before some_function() is called.
	Some amazing function
*** After some_function() is called.


### operator

https://docs.python.org/3/library/operator.html#module-operator

## functools

https://docs.python.org/2/library/functools.html

### generators 

intro:

https://realpython.com/blog/python/introduction-to-python-generators/

not exactly lazy eval



## numpy

numpy is kinda like a sub-dialect of python that data scientists use.

## btw, pandas is kinda functional

most of the time we return a copy of the data so we can chain stuff

In [14]:
import pandas as pd

### Recursion in python

this is a bit annoying in python at first glance

In [15]:
def fact(n, product=1):
    if n <= 0:
        return product
    else:
        return fact(n-1, product*n)    

In [16]:
# this already fails
#fact(976)

In [17]:
# trampolines
# http://www.kylem.net/programming/tailcall.html

class TailCaller(object) :
    def __init__(self, f) :
        self.f = f
    def __call__(self, *args, **kwargs) :
        ret = self.f(*args, **kwargs)
        while type(ret) is TailCall :
            ret = ret.handle()
        return ret
    
class TailCall(object) :
    def __init__(self, call, *args, **kwargs) :
        self.call = call
        self.args = args
        self.kwargs = kwargs
    def handle(self) :
        if type(self.call) is TailCaller :
            return self.call.f(*self.args, **self.kwargs)
        else :
            return self.call(*self.args, **self.kwargs)

In [18]:
def tailcall(f) :
    def _f(*args, **kwargs) :
        return TailCall(f, *args, **kwargs)
    return _f

In [19]:
@TailCaller
def fact(n, r=1) :
    if n <= 1 :
        return r
    else :
        return tailcall(fact)(n-1, n*r)

In [20]:
big_number = 1000

In [21]:
%time tco_answer = fact(big_number)

CPU times: user 3.15 ms, sys: 1.55 ms, total: 4.7 ms
Wall time: 3.56 ms


In [22]:
# still will blow stack
def while_fact(n, product=1):
    while n > 1:
        return while_fact(n-1, product*n)
    return product

In [23]:
# tco_answer

In [24]:
# the following is nasty as we mutate state (see product *= and n -= lines) 
# BUT we can transfer this code easily to other languages 
# and it's almost recursive in that it looks similar to that above. remember practicality counts too!
def pythonic_fact(n, product=1):
    while n > 1:
        product = product * n
        n = n - 1
    return product 

In [25]:
%time answer = pythonic_fact(big_number)

CPU times: user 441 µs, sys: 117 µs, total: 558 µs
Wall time: 481 µs


In [26]:
# answer

In [27]:
tco_answer == answer

True

## Curry in python

return function till all args are collected

In [28]:
# http://stackoverflow.com/questions/9458271/currying-decorator-in-python
# from Julien Palard
def curry_for_breakfast(func):
    def curried(*args, **kwargs):
        if len(args) + len(kwargs) >= func.__code__.co_argcount:
            return func(*args, **kwargs)
        return (lambda *args2, **kwargs2:
                curried(*(args + args2), **dict(kwargs, **kwargs2)))
    return curried

In [29]:
@curry_for_breakfast
def sum5(a, b, c, d, e):
    return a + b + c + d + e

In [30]:
sum5(1)(2)(3)(4)(5)

15

In [31]:
sum5(1, 2, 3)(4, 5)

15

In [32]:
summed2 = sum5(1,2)
summed2(3,4,5)

15

In [33]:
# another implementation
from functools import wraps

def curry_for_lunch(func):
    @wraps(func)
    def curried(*args, **kwargs):
        if len(args) + len(kwargs) >= func.__code__.co_argcount:
            return func(*args, **kwargs)

        @wraps(func)
        def new_curried(*args2, **kwargs2):
            return curried(*(args + args2), **dict(kwargs, **kwargs2))

        return new_curried

    return curried

In [34]:
@curry_for_lunch
def sum3(a, b, c):
    return a + b + c*2

In [35]:
sum3(1)(2)(0)

3

In [36]:
sum3(c=0,a=1,b=2)

3

In [37]:
# we return a function until all args have been supplied
s2 = sum3(1,2)
s2

<function __main__.sum3>

In [38]:
s2(3)

9

In [39]:
s2 = sum3(c=0,b=1)

In [40]:
s2(2)

3

In [41]:
s2(1)

2

### logging example

Currying can be used to clean up functions that require lots of arguements. For example,
if some of the args inputs are the same in many situations, they can be preset as in the following logging example inspired by this blog:

https://templecoding.com/blog/2016/04/13/functional-javascript-introduction-and-currying/

NB: cannot use pythonic optional args in curried applications.

In [42]:
@curry_for_breakfast
def topik_logger(e, msg):
    return e + msg

In [43]:
warning_err = topik_logger(e='Warning error: ')
fatal_err = topik_logger(e='Fatal error: ')
debug_err = topik_logger(e='Debug: ')

In [44]:
warning_err(msg='taking a long time')

'Warning error: taking a long time'

In [45]:
warning_err(msg='slow')

'Warning error: slow'

In [46]:
fatal_err(msg='wrong result!')

'Fatal error: wrong result!'

In [47]:
fatal_err(msg='seg fault')

'Fatal error: seg fault'

In [48]:
debug_err(msg='line is 5')

'Debug: line is 5'

### Data science example

Finding optimal model parameters and training data takes some experimentation. The following illustrates how logging for this process can be cleaned up with currying.

We have one function for logging, but return functions until we fill out the arguments as we go along in the training process. Additional logging like cross validation results could also be added.

In [49]:
@curry_for_breakfast
def model_logger(dataset_name, parameters, train_time, train, test):
    print('dataset name: ', dataset_name, 
          'parameters: ', parameters,
          'training time: ', train_time,
          'train score: ', train, 
          'test score: ', test)
    return 

In [50]:
# set the training set data
aug1 = model_logger('augmented 1x')

In [51]:
for p in [1,2,3]:
    # log the parameters
    plog = aug1(p)
    # we'd now make a model, then store results, time, training and test err
    train_time = 10
    train_score = 95
    test_score = 94
    # finish our log
    plog(train_time, train_score + p, test_score + p)

dataset name:  augmented 1x parameters:  1 training time:  10 train score:  96 test score:  95
dataset name:  augmented 1x parameters:  2 training time:  10 train score:  97 test score:  96
dataset name:  augmented 1x parameters:  3 training time:  10 train score:  98 test score:  97


## Memoization in python

memoization = memorization

https://wiki.python.org/moin/PythonDecoratorLibrary

In [52]:
import functools
def memoize(obj):
    cache = obj.cache = {} 
    
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
           cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

In [53]:
@memoize
def mem_fact(n, product=1):
    while n > 1:
        product = product * n
        n = n - 1
    return product 

In [54]:
%time mem_fact(1000)

CPU times: user 583 µs, sys: 141 µs, total: 724 µs
Wall time: 632 µs


4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

In [55]:
%time mem_fact(1000)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11 µs


4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

#### memoization discussion

* memory can explode
* time can increase as cache grows

## Functional composition

and then function, eg with
double and multiply

In [56]:
def and_then(n, fn): return fn(n)

In [57]:
def multiply(a): return a * 2

In [58]:
def add(a): return a + 2

In [59]:
def divide(a): return a / 2

In [60]:
and_then(multiply(10), add)

22

In [61]:
and_then(add(10), multiply)

24

In [62]:
def compose(*functions):
    return reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [63]:
add_and_multiply = compose(add, multiply)
add_and_multiply(10)

22

In [64]:
multiply_and_add = compose(multiply, add)
multiply_and_add(10)

24

In [65]:
mad = compose(multiply, add, divide)
mad(10)

14